In [31]:
import torch
from transformers import  AutoTokenizer, AlbertForSequenceClassification
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.utils.data import DataLoader
import torch.utils.data as utils
from transformers import get_scheduler
from tqdm.auto import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [32]:
tokenizer = AutoTokenizer.from_pretrained("voidful/albert_chinese_base")
model = AlbertForSequenceClassification.from_pretrained("voidful/albert_chinese_base", problem_type="multi_label_classification")


Some weights of the model checkpoint at voidful/albert_chinese_base were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at voidful/albert_chinese_base and are newly initialized: ['classifier.bias', 'c

In [33]:
tmp_lst = []

for i in ['danmu']:
    with open('{}_token_main.pkl'.format(i), 'rb') as f:
        tmp = pickle.load(f)
        tmp_lst.append(tmp)
    with open('{}_dist_main.pkl'.format(i), 'rb') as f:
        tmp = pickle.load(f)
        tmp_lst.append(tmp)
        
danmu_token, danmu_dist= tmp_lst

In [34]:
inputs = tokenizer(danmu_token, return_tensors="pt",max_length=64,padding='max_length',truncation='longest_first')

In [35]:
print(inputs['input_ids'].shape)
def getlabels(dist_lst):
    tmp = np.zeros_like(dist_lst)
    for i, row in enumerate(dist_lst):
        idx = np.argmax(row)
        tmp[i][idx] = 1
    dist_lst = tmp
    output = torch.from_numpy(np.array(dist_lst).astype(np.float32)).to(device=device)
    return output

torch.Size([337392, 64])


In [36]:
labels = getlabels(danmu_dist)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(inputs['input_ids'], labels, test_size=0.1, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
inputDim, outputDim = X_train.shape[1], y_train.shape[1]

In [38]:
num_labels = 21
length = X_train.shape
inputs_xtrain = {}
inputs_xtrain['input_ids'] = X_train
inputs_xtrain['token_type_ids'] = torch.zeros(X_train.shape,dtype=torch.long)
inputs_xtrain['attention_mask'] = torch.ones(X_train.shape,dtype=torch.long)
inputs_xval = {}
inputs_xval['input_ids'] = X_val
inputs_xval['token_type_ids'] = torch.zeros(X_val.shape,dtype=torch.long)
inputs_xval['attention_mask'] = torch.ones(X_val.shape,dtype=torch.long)

In [39]:
train_dataset = utils.TensorDataset(X_train,y_train)
val_dataset = utils.TensorDataset(X_val,y_val)

In [40]:
train_dataloader = DataLoader(train_dataset, shuffle=True,batch_size=128)
num_epochs = 20
num_training_steps = num_epochs * len(train_dataloader)
model = AlbertForSequenceClassification.from_pretrained("voidful/albert_chinese_base", num_labels=num_labels)
model.to(device)
optimizer = AdamW(model.parameters(),lr=5e-5)
checkpoint = torch.load(PATH,map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
progress_bar = tqdm(range(num_training_steps))
best = checkpoint['acc']
for epoch in range(num_epochs):
    model.train()
    total = 0
    print("epoch",epoch)
    for x,y in train_dataloader:
        x=x.to(device)
        y=y.to(device)
        inputs_xtrain = {}
        inputs_xtrain['input_ids'] = x
        inputs_xtrain['token_type_ids'] = torch.zeros(x.shape,dtype=torch.long).to(device)
        inputs_xtrain['attention_mask'] = torch.ones(x.shape,dtype=torch.long).to(device)
        outputs = model(**inputs_xtrain,labels=y)
        predicted_class_id = outputs.logits.argmax(axis=1).detach().cpu().numpy()
        labels = y.detach().cpu().numpy()
        labels = np.argmax(labels,axis=1)
        correct = np.sum(predicted_class_id==labels)
        total += correct
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    PATH = "model.pt"
    acc = total/y_train.shape[0]
    if best < acc:
        best = acc
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'acc': acc,
                }, PATH)
    print("train accurarcy:",total/y_train.shape[0])
    model.eval()
    total = 0
    eval_dataloader = DataLoader(val_dataset, batch_size=128)
    for x,y in eval_dataloader:
        with torch.no_grad():
            inputs_val = {}
            inputs_val['input_ids'] = x.to(device)
            inputs_val['token_type_ids'] = torch.zeros(x.shape,dtype=torch.long).to(device)
            inputs_val['attention_mask'] = torch.ones(x.shape,dtype=torch.long).to(device)
            logits = model(**inputs_val).logits
            predicted_class_id = logits.argmax(axis=1).detach().cpu().numpy()
            labels = y.detach().cpu().numpy()
            labels = np.argmax(labels,axis=1)
            correct = np.sum(predicted_class_id==labels)
            total += correct
    print("valid accurarcy:",total/y_val.shape[0])

Some weights of the model checkpoint at voidful/albert_chinese_base were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at voidful/albert_chinese_base and are newly initialized: ['classifier.bias', 'c

  0%|          | 0/47460 [00:00<?, ?it/s]

epoch 0
train accurarcy: 0.6201309393647991
valid accurarcy: 0.36733847065797276
epoch 1
train accurarcy: 0.6325629338848419
valid accurarcy: 0.3663307646710136
epoch 2
train accurarcy: 0.6507976235954316
valid accurarcy: 0.37154712507409604
epoch 3
train accurarcy: 0.6733299961798375
valid accurarcy: 0.3759928867812685
epoch 4
train accurarcy: 0.6953123970861381
valid accurarcy: 0.36751630112625966
epoch 5
train accurarcy: 0.7180193115803617
valid accurarcy: 0.37439241256668643
epoch 6
train accurarcy: 0.7377952392870786
valid accurarcy: 0.37083580320094844
epoch 7
train accurarcy: 0.7584010643763255
valid accurarcy: 0.37095435684647304
epoch 8
train accurarcy: 0.7770408230474359
valid accurarcy: 0.3668642560758743
epoch 9
train accurarcy: 0.795031812732997
valid accurarcy: 0.3660936573799644
epoch 10
train accurarcy: 0.8130820808030245
valid accurarcy: 0.3653230586840545
epoch 11
train accurarcy: 0.8303781960928958
valid accurarcy: 0.363841138114997
epoch 12
train accurarcy: 0.846775

In [41]:
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
acc = checkpoint['acc']
test_dataset = utils.TensorDataset(X_test,y_test)
model.eval()
total = 0
test_dataloader = DataLoader(test_dataset, batch_size=128)
for x,y in test_dataloader:
    with torch.no_grad():
        inputs_test = {}
        inputs_test['input_ids'] = x.to(device)
        inputs_test['token_type_ids'] = torch.zeros(x.shape,dtype=torch.long).to(device)
        inputs_test['attention_mask'] = torch.ones(x.shape,dtype=torch.long).to(device)
        logits = model(**inputs_test).logits
        predicted_class_id = logits.argmax(axis=1).detach().cpu().numpy()
        labels = y.detach().cpu().numpy()
        labels = np.argmax(labels,axis=1)
        correct = np.sum(predicted_class_id==labels)
        total += correct
print(total/y_test.shape[0])

0.35376407824540607
